In [3]:
pip install langchain langgraph duckduckgo-search pydantic langchain-openai langchain-community

In [10]:
from typing import TypedDict, Union

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools import DuckDuckGoSearchRun

# LangGraph imports
from langgraph.graph import Graph

# --------------------------
# Define State Schema
# --------------------------
class State(TypedDict):
    question: str
    next_step: Union[str, None]
    response: Union[str, None]

# --------------------------
# Set up LLM
# --------------------------
from google.colab import userdata
OPENAI_API_KEY = userdata.get("OpenAI-key")

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-4o",
    temperature=0
)

# --------------------------
# Tool for General Knowledge
# --------------------------
search_tool = DuckDuckGoSearchRun()

# --------------------------
# Math Chain
# --------------------------
math_prompt = PromptTemplate.from_template("Solve the following math problem: {question}")
math_chain = math_prompt | llm | StrOutputParser()

# --------------------------
# Router Chain
# --------------------------
router_prompt = PromptTemplate.from_template(
    "Given the question: '{question}', decide whether to use 'Math' or 'General Knowledge'. "
    "Respond with only one word: Math or General"
)
router_chain = router_prompt | llm | StrOutputParser()
def route_logic(state: State):
    next_step = state.get("next_step", "General")
    return "math" if next_step == "Math" else "general"

# --------------------------
# Define Nodes
# --------------------------
def router_node(state: State) -> State:
    result = router_chain.invoke({"question": state["question"]})
    return {
        "question": state["question"],
        "next_step": result.strip()
    }

def math_node(state: State) -> State:
    result = math_chain.invoke({"question": state["question"]})
    return {
        "question": state["question"],
        "response": result
    }

def search_node(state: State) -> State:
    response = search_tool.invoke(state["question"])
    return {
        "question": state["question"],
        "response": response
    }

# --------------------------
# Build the Graph
# --------------------------



# --------------------------
# LangGraph Router Agent
# --------------------------
workflow = Graph()
# Add nodes
workflow.add_node("router", router_node)
workflow.add_node("math", math_node)
workflow.add_node("general", search_node)
# Entry point
workflow.set_entry_point("router")
workflow.add_conditional_edges(
    "router",
    route_logic,
    {
        "math": "math",
        "general": "general"
    }
)

# End after either node completes
workflow.add_edge("math", "__end__")
workflow.add_edge("general", "__end__")

# Compile graph
app = workflow.compile()




# --------------------------
# Run the App
# --------------------------
if __name__ == "__main__":
    # Test with a general knowledge question
    inputs: State = {"question": "What is the population of Tokyo?"}
    result = app.invoke(inputs)
    print("Answer (Tokyo):", result["response"])

    # Test with a math question
    inputs_math: State = {"question": "What is 123 * 456?"}
    result_math = app.invoke(inputs_math)
    print("Answer (Math):", result_math["response"])

Answer (Tokyo): The demography of Tokyo is analysed by the Tokyo Metropolitan Government and data is produced for each of the Special wards of Tokyo, the Western Tokyo and the Tokyo Islands, and for all of Tokyo prefecture as a whole. [1] Statistical information is produced about the size and geographical breakdown of the population, the number of people entering and leaving country and the number of people ... How much population does Tokyo have? The current metro area population of Tokyo in 2023 is 37,194,000, a 0.21% decline from 2022. The metro area population of Tokyo in 2022 was 37,274,000, a 0.18% decline from 2021. Tokyo, city and capital of Tokyo 'to' (metropolis) and of Japan. It is located at the head of Tokyo Bay on the Pacific coast of central Honshu. ... Tokyo's population exceeded one million in the late 19th century, and as Japan's political, economic, and cultural centre it became one of the world's most populous cities in the 20th ... That mega-city is of course Tokyo